In [1]:
# %matplotlib notebook
# %load_ext autoreload
# %autoreload 2

import os
import random

from operator import itemgetter

import cv2
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

from albumentations import (
    CLAHE,
    Blur,
    OneOf,
    Compose,
    RGBShift,
    GaussNoise,
    RandomGamma,
    RandomContrast,
    RandomBrightness,
)

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.transforms import HueSaturationValue
from albumentations.augmentations.transforms import Normalize

from trainer import Trainer, hooks, configuration
from detector import Detector
from trainer.utils import patch_configs
from trainer.utils import setup_system
from detection_loss import DetectionLoss
from trainer.encoder import (
    DataEncoder,
    decode_boxes,
    encode_boxes,
    generate_anchors,
    generate_anchor_grid,
)
from trainer.metrics import APEstimator
from trainer.datasets import ListDataset
# from trainer.data_set_downloader import DataSetDownloader 
from trainer.matplotlib_visualizer import MatplotlibVisualizer

/home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
dataloader_config, trainer_config = patch_configs(epoch_num_to_set=10, batch_size_to_set=1)

In [3]:
dataset_config = configuration.DatasetConfig(
    root_dir="../../../Datasets/Road_Scene_Object_Detection",
    train_transforms=[
        RandomBrightness(p=0.5),
        RandomContrast(p=0.5),
        OneOf([
            RandomGamma(),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50),
            RGBShift()
        ],
            p=1),
        OneOf([Blur(always_apply=True), GaussNoise(always_apply=True)], p=1),
        CLAHE(),
        Normalize(),
        ToTensorV2()
    ]
)

In [4]:

dataset_train = ListDataset(
    root_dir=dataset_config.root_dir,
    data_dir = 'export',
    list_file='annots_converted_train.txt',

    classes=[
        "__background__",
        "biker",
        "car",
        "pedestrian",
        "trafficLight",
        "trafficLight-Green",
        "trafficLight-GreenLeft",
        "trafficLight-Red",
        "trafficLight-RedLeft",
        "trafficLight-Yellow",
        "trafficLight-YellowLeft",
        "truck"
    ],
    mode='train',
    transform=Compose(dataset_config.train_transforms),
    input_size=300
)

loader_train = DataLoader(
    dataset=dataset_train,
    batch_size=dataloader_config.batch_size,
    shuffle=True,
    collate_fn=dataset_train.collate_fn,
    num_workers=dataloader_config.num_workers,
    pin_memory=True
)

In [5]:
iterator = iter(loader_train)
one_batch = next(iterator)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/media/rahul/a079ceb2-fd12-43c5-b844-a832f31d5a39/Projects/autonomous_cars/Object_Detector_for_road/SSD_Detector_for_road_training/SSD_Trainer/trainer/datasets.py", line 82, in __getitem__
    img, boxes = resize(img, boxes, (size, size))
  File "/media/rahul/a079ceb2-fd12-43c5-b844-a832f31d5a39/Projects/autonomous_cars/Object_Detector_for_road/SSD_D